## Crear Dataset en el formato necesario

In [2]:
import os
import shutil

from_folder = "SoccerNet/calibration"
to_folder = "dataset-det2/images"

# Crear la carpeta destino si no existe
os.makedirs(to_folder, exist_ok=True)

# Iterar sobre cada subcarpeta
dirs = os.listdir(from_folder)
for dir_name in dirs:
    subfolder_path = os.path.join(from_folder, dir_name)
    to_folder_path = os.path.join(to_folder, dir_name)
    if not os.path.isdir(subfolder_path):
        continue  # Saltar si no es carpeta
    
    if not os.path.exists(to_folder_path):
        os.makedirs(to_folder_path)

    # Listar todas las imágenes en la subcarpeta
    images = [img for img in os.listdir(subfolder_path) if img.endswith(".jpg") or img.endswith(".png")]

    # Copiar las imágenes a la carpeta destino
    for img in images:
        src_path = os.path.join(subfolder_path, img)
        dst_path = os.path.join(to_folder_path, img)
        shutil.copy(src_path, dst_path)

print("Imágenes copiadas correctamente.")

Imágenes copiadas correctamente.


In [3]:
import os
import sys
import json
import numpy as np
from_folder = "SoccerNet/calibration"
labels_folder = "dataset-det2/labels"

# Función auxiliar para obtener el bounding box
def get_bbox_from_points(points):
    xs = [max(0, min(1, p["x"])) for p in points]
    ys = [max(0, min(1, p["y"])) for p in points]
    x_min, x_max = min(xs), max(xs)
    y_min, y_max = min(ys), max(ys)
    x_center = (x_min + x_max) / 2
    y_center = (y_min + y_max) / 2
    width = x_max - x_min
    height = y_max - y_min
    #si tiene 2 puntos calculamos el angulo
    pos = 0 #0 izquierda, 1 derecha
    if len(points) == 2:
        x1, y1 = points[0]["x"], points[0]["y"]
        x2, y2 = points[1]["x"], points[1]["y"]

        # Ordenar: y1 debe ser la parte inferior, y2 la parte superior
        if y1 < y2:
            x1, y1, x2, y2 = x2, y2, x1, y1

        # Si la parte superior (x2) está más a la derecha que la inferior (x1)
        if x2 > x1:
            pos = 1  # inclinación hacia la derecha
        else:
            pos = 2  # inclinación hacia la izquierda
    return x_center, y_center, width, height, pos

# Crear la carpeta destino si no existe
os.makedirs(labels_folder, exist_ok=True)

# Iterar sobre cada subcarpeta
dirs = os.listdir(from_folder)
for dir_name in dirs:
    subfolder_path = os.path.join(from_folder, dir_name)
    to_folder_path = os.path.join(labels_folder, dir_name)
    if not os.path.isdir(subfolder_path):
        continue  # Saltar si no es carpeta
    
    if not os.path.exists(to_folder_path):
        os.makedirs(to_folder_path)

    #listar todos los json
    jsons = [json for json in os.listdir(subfolder_path) if json.endswith(".json")]

    for j in jsons:
        #si el nombre del archivo no es numérico, saltar
        if not j[:-5].isdigit():
            print(f"Nombre de archivo no numérico: {j}")
            continue
        with open(os.path.join(subfolder_path, j), "r") as f:
            try:
                data = json.load(f)
            except Exception as e:
                print(f"Error al cargar el archivo JSON {j}: {e}")
                continue
            label_txt = os.path.join(to_folder_path, j.replace(".json", ".txt"))
            with open(label_txt, "w") as label_file:
                for label, points in data.items():
                    x_c, y_c, w, h, pos = get_bbox_from_points(points)
                    label_file.write(f"{pos} {x_c} {y_c} {w} {h}\n")
print("Etiquetas generadas correctamente.")

Nombre de archivo no numérico: match_info.json
Nombre de archivo no numérico: match_info.json
Nombre de archivo no numérico: match_info.json
Etiquetas generadas correctamente.


# Finetunear Modelo

In [ ]:
from ultralytics import YOLO

# Cargar un modelo base
model = YOLO('yolov8n_line_detail.pt')  
# Entrenar (fine-tune) con tus datos
model.train(data='line_dataset_detail2.yaml', epochs=50, imgsz=640, batch=16, device="cuda", fliplr=0.0,
    flipud=0.0)  # Ajusta los parámetros según tus necesidades

In [5]:
#guardar el modelo
model.save("yolo11n_inclinacion.pt")